##**Welcome to this notebook in whcih we will  fine_tune the large language Model bert for named entity recognition task**

###1- importing Libaries

In [2]:
!pip install transformers torch datasets seqeval


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=a23b29ed1a7582c6c7fe2afcf8b0889879bbbb99ac730f053ecbdd7e4c96eb9f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency 

##2-loading the dataset from Hugging Face's datasets library.

In [3]:
pip install transformers datasets torch scikit-learn


In [ ]:
from datasets import load_dataset

# Load the CoNLL-2003 dataset
dataset = load_dataset("conll2003")


In [ ]:
from datasets import load_dataset

# Load the CoNLL-2003 dataset
dataset = load_dataset("conll2003")

# Display some examples from the dataset as reconstructed raw sentences
for i in range(3):  # Adjust range to see more examples
    print(f"Example {i + 1}")

    # Join tokens to form the sentence
    sentence = " ".join(dataset["train"][i]["tokens"])
    ner_tags = dataset["train"][i]["ner_tags"]

    # Convert NER tag IDs to actual labels
    label_list = dataset["train"].features["ner_tags"].feature.names
    labels = [label_list[tag] for tag in ner_tags]

    # Print the raw sentence
    print("Sentence:", sentence)

    # Print the NER labels for each word in the sentence
    print("NER Labels:", labels)
    print("\n" + "="*50 + "\n")


Example 1
Sentence: EU rejects German call to boycott British lamb .
NER Labels: ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


Example 2
Sentence: Peter Blackburn
NER Labels: ['B-PER', 'I-PER']


Example 3
Sentence: BRUSSELS 1996-08-22
NER Labels: ['B-LOC', 'O']




##3.Preprocess the Data





###We need to tokenize the input text and align the labels with the tokenized words. BERT’s tokenizer splits words into subwords, so it’s crucial to map each subword to the correct label.

In [ ]:
from transformers import AutoTokenizer

# Load BERT tokenizer
model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Define label names and label map
label_list = dataset['train'].features['ner_tags'].feature.names
label_map = {i: label for i, label in enumerate(label_list)}

# Helper function to align labels with tokenized inputs
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding="max_length",
        max_length=128
    )
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        aligned_labels = []
        previous_word_id = None
        for word_id in word_ids:
            if word_id is None:
                aligned_labels.append(-100)  # Ignore the special tokens
            elif word_id != previous_word_id:
                aligned_labels.append(label[word_id])  # First subword
            else:
                aligned_labels.append(-100)  # Ignore subsequent subwords
            previous_word_id = word_id
        labels.append(aligned_labels)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

# Tokenize and align labels for train and validation sets
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

####shwoing examples of the tokenized dataset

In [ ]:
# Display examples of tokenized inputs with aligned labels, skipping padding tokens
for i in range(3):  # Adjust range to see more examples
    print(f"Example {i + 1}")

    # Retrieve input IDs and aligned labels for each example
    input_ids = tokenized_datasets["train"][i]["input_ids"]
    labels = tokenized_datasets["train"][i]["labels"]

    # Convert input IDs to tokens
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    aligned_labels = [label_map[label] if label != -100 else "IGNORE" for label in labels]

    # Display tokens with their aligned labels, ignoring padding
    for token, label in zip(tokens, aligned_labels):
        if token == "[PAD]":
            break  # Stop at the first padding token
        print(f"{token:10} - {label}")
    print("\n" + "="*50 + "\n")


Example 1
[CLS]      - IGNORE
EU         - B-ORG
rejects    - O
German     - B-MISC
call       - O
to         - O
boycott    - O
British    - B-MISC
la         - O
##mb       - IGNORE
.          - O
[SEP]      - IGNORE


Example 2
[CLS]      - IGNORE
Peter      - B-PER
Blackburn  - I-PER
[SEP]      - IGNORE


Example 3
[CLS]      - IGNORE
BR         - B-LOC
##US       - IGNORE
##SE       - IGNORE
##LS       - IGNORE
1996       - O
-          - IGNORE
08         - IGNORE
-          - IGNORE
22         - IGNORE
[SEP]      - IGNORE




In [ ]:
from transformers import AutoModelForTokenClassification

# Load BERT model for token classification with the number of labels in the dataset
model_checkpoint = "bert-base-cased"  # Pretrained BERT model checkpoint
num_labels = len(label_list)  # Total number of labels, derived from CoNLL-2003

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels
)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


###4.Define Training Arguments

In [ ]:
from transformers import TrainingArguments

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


###Step 6: Define the Trainer


The Trainer class in transformers handles the training loop. We’ll also add a metric to evaluate our model.

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.5 MB/s eta 0:00:00


In [ ]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=0c5d5c06d1b52d8eb6d40ca128ae0730591a52489cf6e79dd3c36354e7475072
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
import evaluate
import numpy as np
from transformers import Trainer, DataCollatorForTokenClassification

# Load the seqeval metric for NER using the evaluate library
metric = evaluate.load("seqeval")

# Function to compute evaluation metrics
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (-100) from predictions and labels
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }


In [ ]:
data_collator = DataCollatorForTokenClassification(tokenizer)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

###7.train the model

In [ ]:
# Train the model
trainer.train()


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.067900,0.043784,0.934484,0.939522,0.936996,0.989634
2,0.013900,0.043055,0.945320,0.949461,0.947386,0.991037


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.067900,0.043784,0.934484,0.939522,0.936996,0.989634
2,0.013900,0.043055,0.945320,0.949461,0.947386,0.991037
3,0.005700,0.043721,0.945263,0.951314,0.948279,0.991349


TrainOutput(global_step=5268, training_loss=0.044353134758118466, metrics={'train_runtime': 1269.4959, 'train_samples_per_second': 33.181, 'train_steps_per_second': 4.15, 'total_flos': 2751824963545344.0, 'train_loss': 0.044353134758118466, 'epoch': 3.0})

In [ ]:
from transformers import pipeline

# Create a NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Example text
example_text = "Chris will be moving to New York City."

# Get predictions
predictions = ner_pipeline(example_text)

# Print predictions
for prediction in predictions:
    print(f"Entity: {prediction['word']}, Score: {prediction['score']}, Entity Label: {prediction['entity']}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Entity: Chris, Score: 0.9969006776809692, Entity Label: LABEL_1
Entity: will, Score: 0.9999566078186035, Entity Label: LABEL_0
Entity: be, Score: 0.9999598264694214, Entity Label: LABEL_0
Entity: moving, Score: 0.9999583959579468, Entity Label: LABEL_0
Entity: to, Score: 0.9999572038650513, Entity Label: LABEL_0
Entity: New, Score: 0.9994854927062988, Entity Label: LABEL_5
Entity: York, Score: 0.9994685053825378, Entity Label: LABEL_6
Entity: City, Score: 0.9994444251060486, Entity Label: LABEL_6
Entity: ., Score: 0.999956488609314, Entity Label: LABEL_0


In [ ]:
from transformers import pipeline

# Create a NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Example text
example_text = "Chris will be moving to New York City to live with."

# Get predictions
predictions = ner_pipeline(example_text)

# Print only predictions with entity labels not equal to LABEL_0
for prediction in predictions:
    if prediction['entity'] != 'LABEL_0':  # Change 'LABEL_0' to the actual label name if different
        print(f"Entity: {prediction['word']}, Score: {prediction['score']:.4f}, Entity Label: {prediction['entity']}")


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Entity: Chris, Score: 0.9977, Entity Label: LABEL_1
Entity: New, Score: 0.9995, Entity Label: LABEL_5
Entity: York, Score: 0.9995, Entity Label: LABEL_6
Entity: City, Score: 0.9995, Entity Label: LABEL_6
Entity: James, Score: 0.9986, Entity Label: LABEL_1


###8. Evaluate the Model
Evaluate the model on the validation set and print the precision, recall, F1-score, and accuracy.

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()

# Print evaluation results
print("Evaluation results:", eval_results)


Evaluation results: {'eval_loss': 0.04372091591358185, 'eval_precision': 0.9452628054904586, 'eval_recall': 0.9513140161725068, 'eval_f1': 0.9482787573467675, 'eval_accuracy': 0.9913489078970442, 'eval_runtime': 23.8994, 'eval_samples_per_second': 135.986, 'eval_steps_per_second': 17.03, 'epoch': 3.0}
